In [4]:
import pandas as pd
import requests
import datetime
import traceback

In [5]:
def get_update_array(post):
    title = post['title']
    author = post['author']
    created_utc = post['created_utc']
    self_post = post['is_self']
    score = post['score']
    over_18 = post['over_18']
    num_comments = post['num_comments']
    
    if 'is_original_content' not in post:
        is_original_content = None
    else:
        is_original_content = post['is_original_content']
    
    if 'selftext' not in post:
        self_text = ""
    else:
        self_text = post['selftext']
    
    if 'link_flair_text' not in post:
        flair = None
    else:
        flair = post['link_flair_text']
    
    update_array = [title, flair, score, num_comments, author, is_original_content, created_utc, self_post, self_text, over_18]
    return update_array

In [6]:
start_timestamp = int(datetime.datetime.utcnow().timestamp())
url = "https://api.pushshift.io/reddit/submission/search/?score=>0&before={}&after={}&sort_type=score&sort=desc&subreddit=india&limit=1000"
epoch = start_timestamp
two_days = 2*24*60*60
epoch_prev = epoch - two_days
post_counts = 0
total_years = 5
total = 0
for every_two_days in range(total_years*12*15):
    final_url = url.format(str(epoch),str(epoch_prev))
    json_data = requests.get(final_url, headers={'User-Agent': "test reddit app"})
    try:
        data = json_data.json()
    except ValueError:
        print('JSONDecodeError')
    posts = data['data']
    dataset = []
    for post in posts:
        post_counts = post_counts + 1
        update_array = get_update_array(post)
        dataset.append(update_array)
    df = pd.DataFrame(dataset, columns =['title', 'flair', 'score', 'num_comments', 'author', 'is_original_content', 'created_utc', 'self_post', 'self_text', 'over_18'])
    df.to_csv('./reddit_data.csv', mode='a', index=False, header=False)
    epoch = epoch_prev
    epoch_prev = epoch - two_days
    total += int(df.shape[0])
    print(total)